In [22]:
import pandas as pd
import requests
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

Salvando os arquivos já baixados em .csv em variáveis

## Extraindo dados da API da NOAA
Como o dataset está disponível apenas através da API disponibilizada pela NOAA *(National Oceanic and Atmospheric Administration)*, foi preciso registrar um **token** no site da agência, que foi enviado por email, para conseguir acessar a API

In [23]:
#config
token = "WZzNRktcksOxuQRDpDgdTyqlNgVkqtBd"
endpoint = "https://www.ncei.noaa.gov/access/services/data/v1"

params = {
    "dataset": "daily-summaries",
    "stations": "USW00023174",
    "startDate": "2024-03-07",
    "endDate": "2025-10-31",
    "format": "csv",
    "units": "metric"
}

#request
response = requests.get(endpoint, params=params, headers={"token": token})


Salvar o arquivo extraído da API em um .csv na pasta */data*

In [24]:
from pathlib import Path

root = Path.cwd() #sobe um nível (.parent) pra chegar na raiz v 
data_dir = root/ "data" #aponta pra pasta data/

save_csv = data_dir / "LA-wheather-daily-2024-2025.csv" #salva na pasta data/ 

#salva o arquivo extraído da API na pasta
with open (save_csv, "wb") as f:
    f.write(response.content)
    
print("Arquivo salvo em: ", save_csv)

df_global_hourly = pd.read_csv(r"data\LA-wheather-daily-2024-2025.csv")

Arquivo salvo em:  d:\gabri\Documents\IFSP\5_semestre\SPOESTA\projeto\data\LA-wheather-daily-2024-2025.csv


## Análise estatística

### Perguntas que serão respondidas com as análises:  
  
1. A média de crime diário aumenta quando está calor? (mínimo de 25°C)  
2. Qual a probabilidade de acontecer um crime quando chove?  
3. Qual a probabilidade de um crime ocorrer em dias quentes?  
4. A dispersão de crimes muda entre dias quentes e dias frios?  
5. Qual tipo de crime é mais provável de acontecer em dias frios?

In [53]:
#variáveis para os datasets

nibrs_offenses = pd.read_csv(r"data\LAPD_NIBRS_Offenses_Dataset.csv")
#nibrs_victims = pd.read_csv(r"data\LAPD_NIBRS_Victims_Dataset.csv")
wheather = pd.read_csv(r"data\LA-wheather-daily-2024-2025.csv")

C:\Users\gabri\AppData\Local\Temp\ipykernel_10348\3105674097.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  nibrs_offenses = pd.read_csv(r"data\LAPD_NIBRS_Offenses_Dataset.csv")


## Limpeza e organização dos dados - NIBRS Offenses

In [54]:
print(nibrs_offenses.dtypes)

CaseNo                     object
UniqueNIBRNo               object
Date Rptd                  object
Date OCC                   object
Time OCC                    int64
AREA                        int64
AREA NAME                  object
RPT Dist No                object
TotalOffenseCount           int64
Group                      object
NIBR Code                  object
NIBR Description           object
Crime Against              object
Premis Cd                  object
Premis Desc                object
Weapon Used Cd             object
Weapon Desc                object
Status                      int64
Status Desc                object
TotalVictimCount            int64
Victim Shot                object
Domestic Violence Crime    object
Hate Crime                 object
Gang-related Crime         object
Transit-related Crime      object
Homeless-Victim Crime      object
Homeless-Suspect Crime     object
Homeless-Arrestee Crime    object
dtype: object


In [55]:
# alterando o tipo de dado para conseguir iniciar as análises

nibrs_offenses['Date OCC'] = pd.to_datetime(nibrs_offenses['Date OCC'])
nibrs_offenses['Date'] = nibrs_offenses['Date OCC'].dt.date #extrai apenas a data p/ comparar com o outro dataset que só tem yyyy-mm-dd

#criando a coluna de contagem de crimes por dia
daily_crimes = (
    nibrs_offenses.groupby('Date')['TotalOffenseCount'].sum() #agrupa a coluna ['TotalOffenseCount'] pelo critério ('Date') e soma tudo
    .reset_index() #pq 'Date' virou o índice
)
daily_crimes.columns = ['date', 'crime_count'] 

#pd.set_option("display.max_rows", None)
#daily_crimes

#daily_crimes.isna().sum()

C:\Users\gabri\AppData\Local\Temp\ipykernel_10348\678524114.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nibrs_offenses['Date OCC'] = pd.to_datetime(nibrs_offenses['Date OCC'])


## Limpeza e organização dos dados - NIBRS Offenses

In [56]:
print(wheather.dtypes)

STATION          object
DATE             object
ACMC            float64
ACMH            float64
ACSC            float64
ACSH            float64
ADPT            float64
ASLP            float64
ASTP            float64
AWBT            float64
AWDR            float64
AWND            float64
DAEV            float64
DAPR            float64
DASF            float64
DATN            float64
DATX            float64
DAWM            float64
DWPR            float64
EVAP            float64
FMTM            float64
FRGB            float64
FRGT            float64
FRTH            float64
GAHT            float64
MDEV            float64
MDPR            float64
MDSF            float64
MDTN            float64
MDTX            float64
MDWM            float64
MNPN            float64
MXPN            float64
PGTM            float64
PRCP            float64
PSUN            float64
RHAV            float64
RHMN            float64
RHMX            float64
SN01            float64
SN02            float64
SN03            

In [57]:
wheather['DATE'] = pd.to_datetime(wheather['DATE'])
wheather['Date'] = wheather['DATE'].dt.date